# W-net Metrics

In [1]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import os 
import skimage.measure as meas
import nibabel as nib
import natsort
import glob

In [3]:
orig = "/home/ubuntu/volume1/Raw_data/Kspace/Test/*.npy"
hybrid = "../Recs/*.npy"

orig = glob.glob(orig)
hybrid = glob.glob(hybrid)

orig = natsort.natsorted(orig)
hybrid = natsort.natsorted(hybrid)
print(orig[0])
print(hybrid[0])
ref = np.zeros((170*len(orig),256,256))
hyb = np.zeros((170*len(hybrid),256,256))

/home/ubuntu/volume1/Raw_data/Kspace/Test/e14296s4_P09216.7.npy
../Recs/e14296s4_P09216_hybrid.npy


In [4]:
for ii in range(len(hybrid)):
    aux1 = np.load(orig[ii])
    aux1 = np.abs(np.fft.ifft2(aux1[:,:,:,0]+1j*aux1[:,:,:,1])).astype(np.float64)
    ref[ii*170:(ii+1)*170,:,:] = aux1
    aux2 = np.load(hybrid[ii]).astype(np.float64)
    hyb[ii*170:(ii+1)*170,:,:] = aux2

In [6]:
metrics = np.zeros((hyb.shape[0],3))
for ii in range(hyb.shape[0]):  
    metrics[ii,0] = meas.compare_ssim(ref[ii].ravel(),hyb[ii].ravel(),win_size = ref[ii].size-1)
    metrics[ii,1] = meas.compare_nrmse(ref[ii],hyb[ii],'min-max')
    metrics[ii,2] = meas.compare_psnr(ref[ii],hyb[ii],data_range=(ref[ii].max()-ref[ii].min())) 

In [7]:
metrics[:,1] = metrics[:,1]*100
print("Hybrid")
print("%.3f +/- %.3f" %(metrics[:,0].mean(),metrics[:,0].std()))
print("%.3f +/- %.3f" %(metrics[:,1].mean(),metrics[:,1].std()))
print("%.3f +/- %.3f" %(metrics[:,2].mean(),metrics[:,2].std()))

Hybrid
0.978 +/- 0.076
1.827 +/- 1.112
35.543 +/- 3.239
